In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-for-nlp/dataframetest_with_lemas.csv
/kaggle/input/test-for-nlp/dataframe_with_lemas.csv
/kaggle/input/hse-nlp-bootcamp/sample_submission.csv
/kaggle/input/hse-nlp-bootcamp/train_ml.csv
/kaggle/input/hse-nlp-bootcamp/new_test_ml.csv


In [3]:
df = pd.read_csv('/kaggle/input/hse-nlp-bootcamp/train_ml.csv')
df.head()

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,28.06.2019 13:54
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,15.07.2020 14:54
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,04.08.2020 6:38


In [4]:
df.dropna(inplace=True)

In [5]:
df = df.astype({'grades': 'int32'})
df

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,13.12.2016 1:05
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,04.08.2020 6:38
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,19.04.2015 20:11
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,28.03.2019 18:36
...,...,...,...,...
74995,trust,Три года длится эпопея с закрытием кредитной к...,1,26.10.2016 11:03
74996,fk_otkritie,"Являлся клиентом Бинбанка, пока после слияния ...",2,01.02.2020 21:07
74997,sberbank,8 мая 2016 г. мною вносились деньги на банковс...,1,14.06.2016 16:03
74998,uralsib,Здравствуйте! Сегодня 21.04.2020 захожу в моби...,3,21.04.2020 8:24


In [6]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics

import nltk
from nltk.stem.snowball import SnowballStemmer

import matplotlib.pyplot as plt
import seaborn as sns
import re

In [7]:
def cleanText(string):
    """This function deletes all symbols except Cyrilic and Base Latin alphabet,
    stopwords, functional parts of speech. Returns string of words stem."""
    # Common cleaning
    string = string.lower()
    #string = re.sub(r"http\S+", "", string)
    string = str.replace(string,'Ё','е')
    string = str.replace(string,'ё','е')
    prog = re.compile('[А-Яа-яA-Za-z]+')
    words = prog.findall(string.lower())
    
    # Word Cleaning
    ## Stop Words
    stopwords = nltk.corpus.stopwords.words('russian')
    words = [w for w in words if w not in stopwords]
    ## Cleaning functional POS (Parts of Speech)
    functionalPos = {'CONJ', 'PRCL'}
    words = [w for w, pos in nltk.pos_tag(words, lang='rus') if pos not in functionalPos]
    ## Stemming
    stemmer = SnowballStemmer('russian')
    return ' '.join(list(map(stemmer.stem, words)))

In [10]:
!pip install pymorphy2
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 48.2 MB/s eta 0:00:0000:0100:01


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
import nltk
from nltk.stem.snowball import SnowballStemmer
m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=256)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords]

def clean_text(text):
    text = str.replace(text,'Ё','е')
    text = str.replace(text,'ё','е')
    tokens = words_only(text)
    functionalPos = {'CONJ', 'PRCL'}
    tokens = [w for w, pos in nltk.pos_tag(tokens, lang='rus') if pos not in functionalPos]
    lemmas = lemmatize_text(tokens)
    stemmer = SnowballStemmer('russian')
    return ' '.join(remove_stopwords(lemmas))

In [16]:
df1 = df.feeds[:50]
df1.head()

0    Много лет являюсь клиентом этого банка, но пос...
1    Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....
4    Имею потребительский кредит, взятый в Связь-ба...
7    Хочу выразить глубокую благодарность всем сотр...
8    6 марта 2019г. В 10:33 при проходе в метро "Ку...
Name: feeds, dtype: object

In [33]:
%%time
nltk.download('averaged_perceptron_tagger_ru')
text = np.array(df1.values)
text = list(map(clean_text, text))

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


CPU times: user 2.14 s, sys: 11 ms, total: 2.15 s
Wall time: 2.15 s


In [35]:
%%time

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.loc[index, 'text_token'] = clean_text(row['feeds'])

df.head()

100%|██████████| 51476/51476 [38:02<00:00, 22.55it/s]  

CPU times: user 37min 54s, sys: 10.1 s, total: 38min 4s
Wall time: 38min 2s


,bank,feeds,grades,date,text_token
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,16.02.2017 16:10,год являться клиент это банк последний визит п...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,13.12.2016 1:05,г ростов дон ул ленин г час данный офис оказат...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,04.08.2020 6:38,иметь потребительский кредит взять связь банк ...
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,19.04.2015 20:11,хотеть выразить глубокий благодарность весь со...
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,28.03.2019 18:36,март г проход метро курский социальный карта б...


In [36]:
df.to_csv('dataframe_lemas.csv', index=False)

In [37]:
dftest = pd.read_csv('/kaggle/input/test-for-nlp/dataframetest_with_lemas.csv')
dftest

,Unnamed: 0,bank,feeds,date
0,0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53
1,1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19
2,2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46
3,3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48
4,4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41
...,...,...,...,...
17215,17215,sberbank,"21.09.2018 в 15:26:56, я, О-ва К. К., снимала ...",22.09.2018 16:08
17216,17216,homecreditbank,"Наверняка не засчитают, но тем не менее. Нужна...",06.10.2011 9:35
17217,17217,interprombank,"15.08.2019 я, как физ.лицо обратилась в офис н...",21.08.2019 14:13
17218,17218,otpbank,"У банка есть рекламный продукт-игра ""Мои рыбки...",29.05.2020 16:12


In [38]:
for index, row in tqdm(dftest.iterrows(), total=dftest.shape[0]):
    dftest.loc[index, 'text_token'] = clean_text(row['feeds'])

dftest.head()

100%|██████████| 17220/17220 [12:29<00:00, 22.98it/s]


,Unnamed: 0,bank,feeds,date,text_token
0,0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформлять ипотека сбербанк подгрузить необходи...
1,1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткий содержание рекомендовать брать кредит ...
2,2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемый сотрудник служба контрол...
3,3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращаться получение карта зелёный польза сотр...
4,4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратиться отделение банк проспект ленин отдел...


In [62]:
text = df['text_token']
target = df['grades']
X_train, X_test, y_train, y_test = train_test_split(
    text, target, test_size=0.2, stratify=target
)

In [63]:
print('Dim of train:', len(X_train), '\tTarget rate: {:.2f}%'.format(y_train.mean()))
print("Dim of test:", len(X_test), '\tTarget rate: {:.2f}%'.format(y_test.mean()))

Dim of train: 41180 	Target rate: 2.40%
Dim of test: 10296 	Target rate: 2.40%


In [45]:
clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()), # Prod feature: tokenizer=cleanText
            ("classifier", LinearSVC())]
        )

clf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC())])

In [46]:
print(metrics.classification_report(y_test, clf_pipeline.predict(X_test)))
f1_base = metrics.f1_score(y_test, clf_pipeline.predict(X_test))

              precision    recall  f1-score   support

           1       0.77      0.96      0.85      5548
           2       0.25      0.07      0.10      1127
           3       0.08      0.01      0.02       471
           4       0.39      0.07      0.11       304
           5       0.89      0.93      0.91      2846

    accuracy                           0.78     10296
   macro avg       0.48      0.41      0.40     10296
weighted avg       0.70      0.78      0.73     10296



ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].